In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

import pathlib as plb

In [3]:
WORK_DIRECTORY = plb.Path().resolve()
DATA_DIRECTORY = WORK_DIRECTORY.parent / "data"
HH_DIRECTORY = DATA_DIRECTORY / "Form 2"

In [4]:
[x for x in HH_DIRECTORY.iterdir() if x.is_dir()]

[WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2/Region 1 ILOCOS'),
 WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2/Region 10 NORTHERN MINDANAO'),
 WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2/Region 11 DAVAO'),
 WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2/Region 12 SOCCSKSARGEN'),
 WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2/Region 13 NCR'),
 WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2/Region 14 CAR'),
 WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2/Region 15 ARMM'),
 WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2/Region 16 CARAGA'),
 WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2/Region 2 CAGAYAN VALLEY'),
 WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2/Region 3 CENTRAL LUZON'),
 WindowsPath('D:/WORK/ONLINE COURSES/MINI-PROJECTS/psa_cph/data/Form 2

Population Variables of Interest (Barangay-Level):
- Population (nPOP): `groupby(['REG', 'PRV', 'MUN', 'BGY'])['P2'].count()`
- Sex Ratio (rSX): `P3==1 (male) / P3==2 (female) * 100`
- Youth Dependency Ratio (rYD): `P5<15 / (P5>=15 AND P<65) * 100`
- Old-Age Dependency Ratio (rOD): `P5>65 / (P5>=15 AND P<65) * 100`
- Total Age Dependency Ratio (rAD): `YDR + ODR`
- School Attendance Rate (rSA): `P10==1 / nPOP`<sup>1</sup>
- Literacy Rate (rLT): `P11==1 / nPOP`<sup>1</sup>
- Mean Years of Schooling (mYS): see this [2013 UNESCO document](https://uis.unesco.org/sites/default/files/documents/uis-methodology-for-estimation-of-mean-years-of-schooling-2013-en_0.pdf) for the formula
- Overseas Workers per 1,000 People (ptOFW): `P15==1 / nPOP * 1_000`

Household Variables of Interest (Barangay-Level):
- Number of Households (nHH): `groupby(['REG', 'PRV', 'MUN', 'BGY'])['HSN'].count()`
- Building Type:
    - Percent Single House (pcBT1): `B1==1 / nHH`<sup>1</sup>
    - Percent Duplex (pcBT2): `B1==2 / nHH`<sup>1</sup>
    - Percent Multi-unit Residential (pcBTM): `B1==3 / nHH`<sup>1</sup>
    - Percent Commercial/Industrial/Agricultural (Office, Factory, etc) (pcBTC): `B1==4 / nHH`<sup>1</sup>
    - Percent Institutional Living Quarter (pcILQ): `B1==5 / nHH`<sup>1</sup>
    - Percent Other Housing Units (pcOHU): `B1==6 / nHH`<sup>1</sup>
- Construction Material of Roof:
    - Percent Galvanized Iron/Aluminum (pcRMI): `B2==1 / nHH`<sup>1</sup>
    - Percent Concrete/Clay Tile (pcRMC): `B2==2 / nHH`<sup>1</sup>
    - Percent Half Galvanized Iron and Half Concrete (pcRMH): `B2==3 / nHH`<sup>1</sup>
    - Percent Bamboo/Cogon/Nipa/Anahaw (pcRML): `B2==4 / nHH`<sup>1</sup>
    - Percent Asbestos (pcRMA): `B2==5 / nHH`<sup>1</sup>
    - Percent Makeshift/Salvaged/Improvised Materials (pcRMS): `B2==6 / nHH`<sup>1</sup>
    - Percent Trapal (pcRMT): `B2==7 / nHH`<sup>1</sup>
    - Percent Others (pcRM0): `B2==8 / nHH`<sup>1</sup>
- Construction Material of Outer Walls:
    - Percent Concrete/Brick/Stone (pcOWC): `B3==1 / nHH`<sup>2</sup>
    - Percent Wood (pcOWW): `B3==2 / nHH`<sup>2</sup>
    - Percent Half Concrete/Brick/Stone and Half Wood (pcOWH): `B3==3 / nHH`<sup>2</sup>
    - Percent Galvanized Iron/Aluminum (pcOWI): `B3==4 / nHH`<sup>2</sup>
    - Percent Bamboo/Sawali/Cogon/Nipa (pcOWL): `B3==5 / nHH`<sup>2</sup>
    - Percent Asbestos (pcOWA): `B3==6 / nHH`<sup>2</sup>
    - Percent Glass (pcOWG): `B3==7 / nHH`<sup>2</sup>
    - Percent Makeshift/Salvaged/Improvised Materials (pcOWS): `B3==8 / nHH`<sup>2</sup>
    - Percent Trapal (pcOWT): `B3==9 / nHH`<sup>2</sup>
    - Percent Others (pcOW0): `B2==10 / nHH`<sup>2</sup>
    - Percent No Walls (pcOW_): `B2==11 / nHH`<sup>2</sup>
- Fuel for Lighting:
    - Percent Electricity (pcFLE): `H1==1 / nHH`<sup>1</sup>
    - Percent Kerosene (pcFLK): `H1==2 / nHH`<sup>1</sup>
    - Percent Liquefied Petroleum Gas (pcFLG): `H1==3 / nHH`<sup>1</sup>
    - Percent Oil (Vegetable, Animal, and Others) (pcFLO): `H1==4 / nHH`<sup>1</sup>
    - Percent Solar Panel (pcFLSp): `H1==5 / nHH`<sup>1</sup>
    - Percent Solar Lamp (pcFLSl): `H1==6 / nHH`<sup>1</sup>
    - Percent Others (pcFL0): `H1==7 / nHH`<sup>1</sup>
    - Percent None (pcFL_): `H1==0 / nHH`<sup>1</sup>
- Source of Drinking Water Supply:
    - Percent Own Use Faucet, Community Water System (pcDWFo): `H2==1 / nHH`<sup>2</sup>
    - Percent Shared Faucet, Community Water System (pcDWFs): `H2==2 / nHH`<sup>2</sup>
    - Percent Own Use, Tubed/Piped Deep Well (pcDWDo): `H2==3 / nHH`<sup>2</sup>
    - Percent Shared, Tubed/Piped Deep Well (pcDWDs): `H2==4 / nHH`<sup>2</sup>
    - Percent Tubed/Piped Shallow Well (pcDWS): `H2==5 / nHH`<sup>2</sup>
    - Percent Dug Well (pcDWG): `H2==6 / nHH`<sup>2</sup>
    - Percent Protected Spring (pcDWSp): `H2==7 / nHH`<sup>2</sup>
    - Percent Unprotected Spring (pcDWSu): `H2==8 / nHH`<sup>2</sup>
    - Percent Lake/River/Rain (pcDWL): `H2==9 / nHH`<sup>2</sup>
    - Percent Peddler (pcDWP): `H2==10 / nHH`<sup>2</sup>
    - Percent Bottled Water (pcDWB): `H2==11 / nHH`<sup>2</sup>
    - Percent Others (pcDW0): `H2==12 / nHH`<sup>2</sup>
- Source of Cooking Water Supply:
    - Percent Own Use Faucet, Community Water System (pcKWFo): `H3==1 / nHH`<sup>2</sup>
    - Percent Shared Faucet, Community Water System (pcKWFs): `H3==2 / nHH`<sup>2</sup>
    - Percent Own Use, Tubed/Piped Deep Well (pcKWDo): `H3==3 / nHH`<sup>2</sup>
    - Percent Shared, Tubed/Piped Deep Well (pcKWDs): `H3==4 / nHH`<sup>2</sup>
    - Percent Tubed/Piped Shallow Well (pcKWS): `H3==5 / nHH`<sup>2</sup>
    - Percent Dug Well (pcKWG): `H3==6 / nHH`<sup>2</sup>
    - Percent Protected Spring (pcKWSp): `H3==7 / nHH`<sup>2</sup>
    - Percent Unprotected Spring (pcKWSu): `H3==8 / nHH`<sup>2</sup>
    - Percent Lake/River/Rain (pcKWL): `H3==9 / nHH`<sup>2</sup>
    - Percent Peddler (pcKWP): `H3==10 / nHH`<sup>2</sup>
    - Percent Bottled Water (pcKWB): `H3==11 / nHH`<sup>2</sup>
    - Percent Others (pcKW0): `H3==12 / nHH`<sup>2</sup>
- Tenure Status of Housing Unit/Lot:
    - Percent Own or Owner-Like Possession of House and Lot (pcHLo): `H4==1 / nHH`<sup>1</sup>
    - Percent Rent House/Room, including Lot (pcHLr): `H4==1 / nHH`<sup>1</sup>
    - Percent Own House, Rent Lot (pcHLh): `H4==2 / nHH`<sup>1</sup>
    - Percent Own House, Rent-Free Lot with Consent of Owner (pcHLvc): `H4==3 / nHH`<sup>1</sup>
    - Percent Own House, Rent-Free Lot without Consent of Owner (pcHLvw): `H4==4 / nHH`<sup>1</sup>
    - Percent Rent-free House and Lot with Consent of Owner (pcHLfc): `H4==5 / nHH`<sup>1</sup>
    - Percent Rent-free House and Lot without Consent of Owner (pcHLfw): `H4==6 / nHH`<sup>1</sup>
    - Percent Not Applicable (pcHLn): `H4==1 / nHH`<sup>1</sup>
    

###### <sup>1</sup> do not include `X==9 (not reported)`
###### <sup>2</sup> do not include `X==99 (not reported)`